In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [3]:
import netCDF4 as nc
import sys
sys.path.append('/data/tjarniko/mocsy')
import mocsy
import numpy as np

import gsw
import arrow
import glob
import cmocean as cm
import matplotlib.pyplot as plt

from salishsea_tools import (
    nc_tools,
    viz_tools,
    geo_tools,
    tidetools
)

%matplotlib inline

## Purpose of notebook

Here I'm going to first plot your stations on a map, and then take the timeseries of DIC, TA, temp, and salinity that we've built in part B and calculate pH and $\Omega_A$ from it and visualise those as a timeseries. The python package I use for this is called [mocsy](http://ocmip5.ipsl.jussieu.fr/mocsy/), and it's relatively fancy - and installing it on your own computer can be a bit of a pain. If you have access to the Waterhole cluster, you can just use the path to the installation that I listed above. Below I've written a wrapper code for it (oned_moxy). 




#### TJSJ let's make a dict with placenames, colors, and then code a map to visualise the stations

In [5]:
t_nc = nc.Dataset('./FILES_TO_GIVE_EVIE/IEXTRACTED_timeseries/carp_timeseries_2017.nc')
t_str = 'pHOm_timeseries_2017.nc'

FileNotFoundError: [Errno 2] No such file or directory: b'./FILES_TO_GIVE_EVIE/IEXTRACTED_timeseries/carp_timeseries_2017.nc'

In [ ]:
PLACES = {
'Deep Bay': {
    
'lon lat': (-124.7392, 49.4606),

'color': 'red', 'NEMO grid ji': (599, 126),

},

'Southern Baynes': {

'lon lat': (-124.7457, 49.4760),

'color': 'sienna', 'NEMO grid ji': (602, 127),

},

'Northern Baynes': {

'lon lat': (-124.8924, 49.6492),

'color': 'orange', 'NEMO grid ji': (646, 127),

},

'Fanny Bay': {

'lon lat': (-124.8227, 49.5086),

'color': 'gold', 'NEMO grid ji': (614, 120),

},

'Maple Bay': {

'lon lat': (-123.5947, 48.8140),

'color': 'olivedrab', 'NEMO grid ji': (392, 213),

},

'Salt Spring': {

'lon lat': (-123.5513, 48.7993),

'color': 'lightgreen', 'NEMO grid ji': (386, 218),

},

'Nanoose Bay': {

'lon lat': (-124.1359, 49.2609),

'color': 'turquoise', 'NEMO grid ji': (517, 190),

},

'Lasqueti Island': {

# deg E, deg N

'lon lat': (-124.3384, 49.5442),

'color': 'dodgerblue', 'NEMO grid ji': (586, 195),

},

'Main SoG': {

'lon lat': (-123.5832, 49.1177),

'color': 'slategray', 'NEMO grid ji': (450, 253),

},

'Cortes/Marina': {

'lon lat': (-125.0194, 50.0418),

'color': 'royalblue', 'NEMO grid ji': (732, 157),

},

'Lund/Desolation Sound': {

'lon lat': (-124.7666, 49.9804),

'color': 'darkviolet', 'NEMO grid ji': (702, 187),

},

'Mouth of Okeover': {

'lon lat': (-124.8174, 50.0805),

'color': 'pink', 'NEMO grid ji': (726, 192),

},
}

list_places = ['Deep Bay', 'Southern Baynes', 'Northern Baynes', \
               'Fanny Bay', 'Maple Bay', 'Salt Spring', 'Nanoose Bay',\
               'Lasqueti Island', 'Main SoG', 'Cortes/Marina', \
               'Lund/Desolation Sound', 'Mouth of Okeover']

#### map plotting code

In [ ]:
bath = nc.Dataset('/data/tjarniko/MEOPAR/grid/bathymetry_201702.nc')
bath1 = bath.variables['Bathymetry']

def wshed_plot(surfdat_1,tit1,t_cmap,xsize,ysize,v_min1,v_max1,cl1,bigtit):

    fig, ax = plt.subplots(1, 1, figsize=(xsize, ysize), sharey=True)
    cmap = t_cmap
            

    tplt = np.ma.masked_values(surfdat_1,0)
    ax.set_title(tit1,fontsize = 20 )
    v_min = v_min1
    v_max = v_max1
    clabel = cl1

    viz_tools.set_aspect(ax)
    mesh = ax.pcolormesh(tplt, cmap=t_cmap, vmin=v_min, vmax=v_max) 
    ax.set_xlabel('x Index', fontsize = 20 )
    ax.set_ylabel('y Index', fontsize = 20 )
    
    for i in range(0,len(list_places)):
        t_place = list_places[i]
        tj = PLACES[t_place]['NEMO grid ji'][1]
        ti = PLACES[t_place]['NEMO grid ji'][0]
        col = PLACES[t_place]['color']
        pts = ax.scatter(tj,ti,s=105,c=col,marker='o')
        ax.text(tj+5,ti-2,t_place, weight = 'bold', fontsize = 15)

    plt.xlim([0,398])
    plt.ylim([0,898])
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    cmap.set_bad('lightslategray')
    plt.legend(bbox_to_anchor=(1.29, 0.95))
    plt.suptitle(bigtit,fontsize=20)
    
fact = 0.6
#wshed_plot(bath1,'Named Stations for Evie', \
 #              cm.cm.dense,20*fact,30*fact,0,1000,'bathymetry','')

#### TJSJ wrapper code for mocsy

In [ ]:
def oned_moxy(tsal, ttemp, tdic, tta, pres_atm, depth_this):

    size_box = np.shape(tdic)
    size_0 = size_box[0]
    size_1= size_box[1]

    tsra = np.ravel(tsal)
    ttera = np.ravel(ttemp)
    ttara = np.ravel(tta) * 1e-3
    tdra = np.ravel(tdic) * 1e-3
    tzero = np.zeros_like(tsra)
    tpressure = np.zeros_like(tsra)
    tdepth = np.zeros_like(tsra)
    tdepth[:] = depth_this
     
    tpressure[:] = pres_atm
    tzero = tpressure * 0 
        
    tsra_psu = tsra*35/35.16504
    ttera_is = gsw.t_from_CT(tsra,ttera,tzero)

    response_tup = mocsy.mvars(temp=ttera_is, sal=tsra_psu, alk=ttara, dic=tdra, 
                       sil=tzero, phos=tzero, patm=tpressure, depth=tdepth, lat=tzero, 
                        optcon='mol/m3', optt='Tinsitu', optp='m',
                        optb = 'l10', optk1k2='m10', optkf = 'dg', optgas = 'Pinsitu')
    pH,pco2,fco2,co2,hco3,co3,OmegaA,OmegaC,BetaD,DENis,p,Tis = response_tup

    pHr = pH.reshape(size_0,size_1)
    OmAr = OmegaA.reshape(size_0,size_1)
    OmCr = OmegaC.reshape(size_0,size_1)
    pco2r = pco2.reshape(size_0,size_1)
    
    return pHr, OmAr, OmCr, pco2r

#### TJSJ let's make a dict in which to store the OmA and pH timeseries

In [ ]:
PLACES_withdat = {
'Deep Bay': {
'NEMO grid ji': (599, 126),
    'OmA_timeseries':np.zeros([40,365]),
    'OmC_timeseries':np.zeros([40,365]),
    'pH_timeseries':np.zeros([40,365])

},
    
'Southern Baynes': {
'NEMO grid ji': (602, 127),
    'OmA_timeseries':np.zeros([40,365]),
    'OmC_timeseries':np.zeros([40,365]),
    'pH_timeseries':np.zeros([40,365])
},

'Northern Baynes': {
'NEMO grid ji': (646, 127),
    'OmA_timeseries':np.zeros([40,365]),
    'OmC_timeseries':np.zeros([40,365]),
    'pH_timeseries':np.zeros([40,365])},

'Fanny Bay': {
'NEMO grid ji': (614, 120),
    'OmA_timeseries':np.zeros([40,365]),
    'OmC_timeseries':np.zeros([40,365]),
    'pH_timeseries':np.zeros([40,365])},

'Maple Bay': {
'NEMO grid ji': (392, 213),
    'OmA_timeseries':np.zeros([40,365]),
    'OmC_timeseries':np.zeros([40,365]),
    'pH_timeseries':np.zeros([40,365])
},

'Salt Spring': {
'NEMO grid ji': (386, 218),
    'OmA_timeseries':np.zeros([40,365]),
    'OmC_timeseries':np.zeros([40,365]),
    'pH_timeseries':np.zeros([40,365])
},

'Nanoose Bay': {
'NEMO grid ji': (517, 190),
    'OmA_timeseries':np.zeros([40,365]),
    'OmC_timeseries':np.zeros([40,365]),
    'pH_timeseries':np.zeros([40,365])
},

'Lasqueti Island': {
'NEMO grid ji': (586, 195),
    'OmA_timeseries':np.zeros([40,365]),
    'OmC_timeseries':np.zeros([40,365]),
    'pH_timeseries':np.zeros([40,365])
},

'Main SoG': {
'NEMO grid ji': (450, 253),
    'OmA_timeseries':np.zeros([40,365]),
    'OmC_timeseries':np.zeros([40,365]),
    'pH_timeseries':np.zeros([40,365])

},

'Cortes/Marina': {
'NEMO grid ji': (732, 157),
    'OmA_timeseries':np.zeros([40,365]),
    'OmC_timeseries':np.zeros([40,365]),
    'pH_timeseries':np.zeros([40,365])},

'Lund/Desolation Sound': {
'NEMO grid ji': (702, 187),
    'OmA_timeseries':np.zeros([40,365]),
    'OmC_timeseries':np.zeros([40,365]),
    'pH_timeseries':np.zeros([40,365])},

'Mouth of Okeover': {
'NEMO grid ji': (726, 192),
    'OmA_timeseries':np.zeros([40,365]),
    'OmC_timeseries':np.zeros([40,365]),
    'pH_timeseries':np.zeros([40,365])},

}

list_places = ['Deep Bay', 'Southern Baynes', 'Northern Baynes', \
               'Fanny Bay', 'Maple Bay', 'Salt Spring', 'Nanoose Bay',\
               'Lasqueti Island', 'Main SoG', 'Cortes/Marina', \
               'Lund/Desolation Sound', 'Mouth of Okeover']


#### TJSJ let's open the .nc we made in part B, extract the DIC, TA, S, T, variables and run mocsy on them to get pH and Omega A, then store these in the dictionary we just made

In [ ]:
for i in range(0,len(list_places)):
    stn = list_places[i]

    DIC = t_nc[stn]['DIC_timeseries'][:,:]
    TA = t_nc[stn]['TA_timeseries'][:,:]
    TEMP = t_nc[stn]['T_timeseries'][:,:]
    SAL = t_nc[stn]['S_timeseries'][:,:]

    pHr, OmAr, OmCr, pco2r =oned_moxy(SAL, TEMP, DIC, TA, 1, 0)

    PLACES_withdat[stn]['OmA_timeseries'] = OmAr
    PLACES_withdat[stn]['OmC_timeseries'] = OmCr
    PLACES_withdat[stn]['pH_timeseries'] = pHr


In [ ]:


f = nc.Dataset(t_str, 'w')
g = f.createGroup('Deep Bay')
g.createDimension('depths', 40)
g.createDimension('days',365)
ts = g.createVariable('OmA_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Deep Bay']['OmA_timeseries'][:]
ts = g.createVariable('OmC_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Deep Bay']['OmC_timeseries'][:]
ts = g.createVariable('pH_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Deep Bay']['pH_timeseries'][:]


g = f.createGroup('Southern Baynes')
g.createDimension('depths', 40)
g.createDimension('days',365)
ts = g.createVariable('OmA_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Southern Baynes']['OmA_timeseries'][:]
ts = g.createVariable('OmC_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Southern Baynes']['OmC_timeseries'][:]
ts = g.createVariable('pH_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Southern Baynes']['pH_timeseries'][:]


g = f.createGroup('Northern Baynes')
g.createDimension('depths', 40)
g.createDimension('days',365)
ts = g.createVariable('OmA_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Northern Baynes']['OmA_timeseries'][:]
ts = g.createVariable('OmC_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Northern Baynes']['OmC_timeseries'][:]
ts = g.createVariable('pH_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Northern Baynes']['pH_timeseries'][:]


g = f.createGroup('Fanny Bay')
g.createDimension('depths', 40)
g.createDimension('days',365)
ts = g.createVariable('OmA_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Fanny Bay']['OmA_timeseries'][:]
ts = g.createVariable('OmC_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Fanny Bay']['OmC_timeseries'][:]
ts = g.createVariable('pH_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Fanny Bay']['pH_timeseries'][:]


g = f.createGroup('Maple Bay')
g.createDimension('depths', 40)
g.createDimension('days',365)
ts = g.createVariable('OmA_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Maple Bay']['OmA_timeseries'][:]
ts = g.createVariable('OmC_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Maple Bay']['OmC_timeseries'][:]
ts = g.createVariable('pH_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Maple Bay']['pH_timeseries'][:]

g = f.createGroup('Salt Spring')
g.createDimension('depths', 40)
g.createDimension('days',365)
ts = g.createVariable('OmA_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Salt Spring']['OmA_timeseries'][:]
ts = g.createVariable('OmC_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Salt Spring']['OmC_timeseries'][:]
ts = g.createVariable('pH_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Salt Spring']['pH_timeseries'][:]


g = f.createGroup('Nanoose Bay')
g.createDimension('depths', 40)
g.createDimension('days',365)
ts = g.createVariable('OmA_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Nanoose Bay']['OmA_timeseries'][:]
ts = g.createVariable('OmC_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Nanoose Bay']['OmC_timeseries'][:]
ts = g.createVariable('pH_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Nanoose Bay']['pH_timeseries'][:]

g = f.createGroup('Lasqueti Island')
g.createDimension('depths', 40)
g.createDimension('days',365)
ts = g.createVariable('OmA_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Lasqueti Island']['OmA_timeseries'][:]
ts = g.createVariable('OmC_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Lasqueti Island']['OmC_timeseries'][:]
ts = g.createVariable('pH_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Lasqueti Island']['pH_timeseries'][:]

g = f.createGroup('Main SoG')
g.createDimension('depths', 40)
g.createDimension('days',365)
ts = g.createVariable('OmA_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Main SoG']['OmA_timeseries'][:]
ts = g.createVariable('OmC_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Main SoG']['OmC_timeseries'][:]
ts = g.createVariable('pH_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Main SoG']['pH_timeseries'][:]


g = f.createGroup('Cortes/Marina')
g.createDimension('depths', 40)
g.createDimension('days',365)
ts = g.createVariable('OmA_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Cortes/Marina']['OmA_timeseries'][:]
ts = g.createVariable('OmC_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Cortes/Marina']['OmC_timeseries'][:]
ts = g.createVariable('pH_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Cortes/Marina']['pH_timeseries'][:]


g = f.createGroup('Lund/Desolation Sound')
g.createDimension('depths', 40)
g.createDimension('days',365)
ts = g.createVariable('OmA_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Lund/Desolation Sound']['OmA_timeseries'][:]
ts = g.createVariable('OmC_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Lund/Desolation Sound']['OmC_timeseries'][:]
ts = g.createVariable('pH_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Lund/Desolation Sound']['pH_timeseries'][:]


g = f.createGroup('Mouth of Okeover')
g.createDimension('depths', 40)
g.createDimension('days',365)
ts = g.createVariable('OmA_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Mouth of Okeover']['OmA_timeseries'][:]
ts = g.createVariable('OmC_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Mouth of Okeover']['OmC_timeseries'][:]
ts = g.createVariable('pH_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Mouth of Okeover']['pH_timeseries'][:]



f.close()

In [ ]:
print('done, yo')